In [21]:
import json
import pandas as pd

In [22]:
df = pd.read_csv(r"word_list\english\unigram_freq.csv")

# only lowercase word
df["word"] = df["word"].apply(lambda x: str(x).lower())
agg_functions = {"word": "first", "count": "sum"}
df = df.groupby(df["word"], as_index=False).aggregate(agg_functions)

# allowed alphabets
df = df[df["word"].str.isalpha()]

df["word_len"] = df["word"].apply(len)
df = df.sort_values(by=["word"]).reset_index(drop=True)
df.head()

,word,count,word_len
0,a,9081174698,1
1,aa,30523331,2
2,aaa,10243983,3
3,aaaa,1595769,4
4,aaaah,52821,5


In [23]:
df.dtypes

word        object
count        int64
word_len     int64
dtype: object

In [24]:
df.describe()

,count,word_len
count,3.333320e+05,333332.000000
mean,1.764380e+06,7.470345
std,6.629997e+07,2.791021
min,1.271100e+04,1.000000
25%,2.122400e+04,5.000000
50%,4.151900e+04,7.000000
75%,1.365738e+05,9.000000
max,2.313585e+10,38.000000


In [25]:
df[df["word_len"] == 3].describe()

,count,word_len
count,1.297700e+04,12977.0
mean,7.687292e+06,3.0
std,2.457968e+08,0.0
min,1.271100e+04,3.0
25%,4.801600e+04,3.0
50%,1.517290e+05,3.0
75%,5.480400e+05,3.0
max,2.313585e+10,3.0


In [26]:
df.query("'running' in word")

,word,count,word_len
253941,running,72879426,7


In [27]:
difficulties = {"easy": 100, "medium": 200, "hard": 300, "all": 1000}
lengths = [3]

for l in lengths:
    for name, count in difficulties.items():
        df_filtered = (
            df[df["word_len"] == l]
            .sort_values(by=["count"], ascending=False)
            .head(count)
            .reset_index(drop=True)
        )
        with open(rf"vocabs\en\en-len{l}-{name}.json", "w") as f:
            data = df_filtered["word"].sort_values().tolist()
            json.dump(data, f)